In [1]:
import sys
sys.path.append('../')
import matplotlib.pyplot as plt
import numpy as np
import pygmt

from remit.earthvim import PolarityTimescale
from remit.utils.grid import make_dataarray, coeffs2map
from gprm.datasets import Seafloor

from remit.data.models import load_lcs, load_ocean_age_model, load_vis_model
from basis_models import load_vim_models

%load_ext autoreload
%autoreload 2


In [2]:
# A single cell to construct the VIM model
ocean = load_ocean_age_model()

vis = load_vis_model(match=(ocean.lon, ocean.lat, ocean.age))

pol = PolarityTimescale()
polarity_da = make_dataarray(ocean.lon, ocean.lat, pol.Interpolator(ocean.age))
polarity_da.data[np.isnan(ocean.age.data)] = np.nan


In [3]:

lmax = 130
cmax = 50
altitude = 0
showgravity = True

lcs = load_lcs(lmax=lmax)
lcs_rad = lcs.expand(a=lcs.r0).rad
lcs_rad_300 = lcs.expand(a=lcs.r0+300000).rad

model_dict = load_vim_models(
    ['DAH981', 
     'HM05',
     'GK07', 
     'DAH982'], 
    altitude=altitude, lmax=lmax)
model_dict_300 = load_vim_models(
    ['DAH981', 
     'HM05',
     'GK07', 
     'DAH982'], 
    altitude=300000, lmax=lmax)



In [4]:
def plot_mag_map(fig, rad, projection, region, cmax, showgravity=True, frame='WSne'):

    radi = pygmt.grdsample(grid=rad.to_xarray(), spacing='0.05d')
    pygmt.config(COLOR_FOREGROUND='red', COLOR_BACKGROUND='blue')
    pygmt.makecpt(cmap='polar', series='-{:f}/{:f}'.format(float(cmax), float(cmax)), reverse=True)
    radi.to_netcdf('./grids/_tmp.nc')
    fig.grdimage(region=region, projection=projection, 
                 grid='./grids/_tmp.nc', transparency=10)

    pygmt.config(COLOR_FOREGROUND='white', COLOR_BACKGROUND='black')
    pygmt.makecpt(cmap='gray', series='-60/60', reverse=True)
    if showgravity:
        fig.grdimage(region=region, projection=projection,
                     grid='/Users/simon/Data/GMTdata/VGG22/grav_29.1.nc',
                     shading='/Users/simon/Data/GMTdata/VGG22/grav_29.1.nc+a-45+nt2+m0',
                     transparency=50)
    
    fig.plot(data='./gis/Muller_etal_AREPS_2016_C13y.gmt', pen='0.6p,black')
    fig.plot(data='./gis/Muller_etal_AREPS_2016_C34.gmt', pen='0.6p,black')
    fig.plot(data='./gis/Muller_etal_AREPS_2016_M0.gmt', pen='0.6p,black')
    fig.plot(data='./gis/Muller_etal_AREPS_2016_Ridges.gmt', pen='1p,black,-')

    fig.coast(shorelines='0.5p,gray20', resolution='l', area_thresh=5000.,
              region=region, projection=projection, land='gray', transparency=20)
    fig.basemap(frame=['afg',frame], region=region, projection=projection)
    
    pygmt.config(COLOR_FOREGROUND='red', COLOR_BACKGROUND='blue')
    pygmt.makecpt(cmap='polar', series='-{:f}/{:f}'.format(float(cmax), float(cmax)), reverse=True)
    with pygmt.config(FONT_ANNOT_PRIMARY='10p', FONT_LABEL='12p'):
        fig.colorbar(position='JBL+jBL+o0.7c/1.4c+w3.5c/0.33c+h+e', frame=['x+lBr [nT]'], box='+gwhite+p0.2p')


def plot_polarity(fig, polarity_da, projection, region, showgravity=True):
    pygmt.makecpt(cmap='hot', series='-1.5/1.5', reverse=True)
    polarity_da.to_netcdf('./grids/_tmp.nc')
    fig.grdimage(region=region, projection=projection, 
                 grid = './grids/_tmp.nc',
                 interpolation='n', transparency=20)

    pygmt.config(COLOR_FOREGROUND='white', COLOR_BACKGROUND='black')
    pygmt.makecpt(cmap='gray', series='-60/60', reverse=True)
    
    if showgravity:
        fig.grdimage(region=region, projection=projection,
                     grid='/Users/simon/Data/GMTdata/VGG22/grav_29.1.nc',
                     shading='/Users/simon/Data/GMTdata/VGG22/grav_29.1.nc+a-45+nt2+m0',
                     transparency=50)
    
    fig.plot(data='./gis/CGMW_LipsSeamounts.shp', fill='deeppink', pen='0.2p,black', transparency=20)
    fig.coast(shorelines='0.5p,gray20', resolution='l', area_thresh=5000.,
              region=region, projection=projection, land='gray', transparency=20)
    fig.basemap(frame='afg', region=region, projection=projection)


def plot_seafloor_fabric(fig, projection, region, showgravity=True):

    pygmt.config(COLOR_FOREGROUND='white', COLOR_BACKGROUND='black')
    pygmt.makecpt(cmap='gray', series='-60/60', reverse=True)
    
    if showgravity:
        fig.grdimage(region=region, projection=projection,
                     grid='/Users/simon/Data/GMTdata/VGG22/grav_29.1.nc',
                     shading='/Users/simon/Data/GMTdata/VGG22/grav_29.1.nc+a-45+nt2+m0',
                     transparency=50)

    fig.plot(data=Seafloor.MagneticPicks(load=False), style='c0.05c', color='orange', transparency=20)
    fig.plot(data=Seafloor.SeafloorFabric(feature_type='FZ', load=False), pen='0.3p,black')
    fig.plot(data='./gis/CGMW_LipsSeamounts.shp', fill='darkred', pen='0.2p,black', transparency=20)
    fig.plot(data='./gis/Muller_etal_AREPS_2016_Ridges.gmt', pen='0.3p,dodgerblue')

    fig.coast(shorelines='0.5p,gray20', resolution='l', area_thresh=5000.,
              region=region, projection=projection, land='darkgray', transparency=5)
    fig.basemap(frame='afg', region=region, projection=projection)


def PlotPanels(fig, projection, region, figsize, cmax, altitude=0, showgravity=True):
    
    if altitude==300000:
        (LCS_1, 
         model_1, model_2, 
         model_3, model_4) = [lcs_rad_300, 
                              model_dict_300['DAH981']['rad'], 
                              model_dict_300['HM05']['rad'], 
                              model_dict_300['GK07']['rad'], 
                              model_dict_300['DAH982']['rad']] 
    else:
        (LCS_1, 
         model_1, model_2, 
         model_3, model_4) = [lcs_rad, 
                              model_dict['DAH981']['rad'], 
                              model_dict['HM05']['rad'], 
                              model_dict['GK07']['rad'], 
                              model_dict['DAH982']['rad']]
    
    with fig.subplot(nrows=2, ncols=3, figsize=figsize, frame="lrtb", autolabel='+gwhite+pblack', margins='0.15i'):
        with fig.set_panel(panel=0, fixedlabel='LCS'):
            plot_mag_map(fig, LCS_1, projection=projection, region=region, cmax=cmax, 
                         showgravity=showgravity, frame='Wsne')
        with fig.set_panel(panel=3, fixedlabel='Polarity'):
            plot_polarity(fig, polarity_da, projection, region, showgravity=showgravity)
            
        with fig.set_panel(panel=1, fixedlabel='DAH981'):
            plot_mag_map(fig, model_1, projection=projection, region=region, cmax=cmax, 
                         showgravity=showgravity, frame='wsne')
        with fig.set_panel(panel=2, fixedlabel='HM05'):
            plot_mag_map(fig, model_2, projection=projection, region=region, cmax=cmax, 
                         showgravity=showgravity, frame='wsne')
        
        with fig.set_panel(panel=4, fixedlabel='GK07'):
            plot_mag_map(fig, model_3, projection=projection, region=region, cmax=cmax, 
                         showgravity=showgravity, frame='wSne')
        with fig.set_panel(panel=5, fixedlabel='DAH982'):
            plot_mag_map(fig, model_4, projection=projection, region=region, cmax=cmax, 
                         showgravity=showgravity, frame='wSne')

        

In [5]:
projection='A-30/25/?'
region = '-75/0/0/50r'

fig = pygmt.Figure()
PlotPanels(fig, projection, region, figsize=("16i", "7.5i"), cmax=cmax, showgravity=showgravity)
fig.savefig('./figures/CAtlantic_{:d}km_lmax{:d}.png'.format(int(altitude/1000),int(lmax)))
#fig.show(width=1500)


In [6]:
projection='A-30/25/?'
region = '-75/0/0/50r'

fig = pygmt.Figure()
#PlotPanels2(fig, projection, region, figsize=("10i", "20i"), cmax=cmax)
PlotPanels(fig, projection, region, figsize=("16i", "7.5i"), altitude=300000, cmax=6, showgravity=showgravity)
fig.savefig('./figures/CAtlantic_{:d}km_lmax{:d}.png'.format(int(300),int(lmax)))
#fig.show(width=1500)


In [7]:
projection='A-135/35/?'
region = '-170/10/-100/60r'

fig = pygmt.Figure()
PlotPanels(fig, projection, region, figsize=("16i", "9i"), cmax=cmax, showgravity=showgravity)
fig.savefig('./figures/NEPacific_{:d}km_lmax{:d}.png'.format(int(altitude/1000),int(lmax)))
#fig.show(width=1500)


In [8]:
projection='A-170/-55/?'
region = '110/-70/-130/-30r'
    
fig = pygmt.Figure()
PlotPanels(fig, projection, region, figsize=("16i", "8.7i"), cmax=cmax, showgravity=showgravity)
fig.savefig('./figures/SWPacific_{:d}km_lmax{:d}.png'.format(int(altitude/1000.),int(lmax)))
#fig.show(width=1500)

In [9]:
projection='A-15/-20/?'
region = '-75/-50/16/10r'

fig = pygmt.Figure()
PlotPanels(fig, projection, region, figsize=("16i", "10i"), cmax=cmax, showgravity=showgravity)
fig.savefig('./figures/SAtlantic_{:d}km_lmax{:d}.png'.format(int(altitude/1000.),int(lmax)))
#fig.show(width=1500)


In [10]:
projection='A-105/-25/?'
region = '-175/-40/-70/-2r'

fig = pygmt.Figure()
PlotPanels(fig, projection, region, figsize=("16i", "6.1i"), cmax=cmax, showgravity=showgravity)
fig.savefig('./figures/EPacificRise_{:d}km_lmax{:d}.png'.format(int(altitude/1000),int(lmax)))
#fig.show(width=1500)


In [11]:
projection='A115/-45/?'
region = '60/-55/145/-22r'

fig = pygmt.Figure()
PlotPanels(fig, projection, region, figsize=("16i", "7.5i"), cmax=cmax, showgravity=showgravity)
fig.savefig('./figures/SEIR_{:d}km_lmax{:d}.png'.format(int(altitude/1000),int(lmax)))
#fig.show(width=1500)


In [12]:
projection='A80/-5/?'
region = '68/-20/95/12r'

fig = pygmt.Figure()
PlotPanels(fig, projection, region, figsize=("12i", "8.7i"), cmax=cmax)
fig.savefig('./figures/Wharton_{:d}km_lmax{:d}.png'.format(int(altitude/1000),int(lmax)))
#fig.show(width=1500)

In [13]:
projection='A35/-35/?'
region = '10/-60/70/-10r'

fig = pygmt.Figure()
PlotPanels(fig, projection, region, figsize=("16i", "10.5i"), cmax=cmax, showgravity=showgravity)
fig.savefig('./figures/SWIR_{:d}km_lmax{:d}.png'.format(int(altitude/1000),int(lmax)))


In [14]:
projection='A175/20/?'
region = '140/-15/210/50r'

fig = pygmt.Figure()
PlotPanels(fig, projection, region, figsize=("16i", "12i"), cmax=cmax, showgravity=showgravity)
fig.savefig('./figures/WPAC_{:d}km_lmax{:d}.png'.format(int(altitude/1000),int(lmax)))
